In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import with_statement

import collections
import gzip
import os
import re
import random

### Import the Spacy library

[Spacy](http://spacy.io) is an NLP library which is ready to be used in production settings. This library will help us find features for our data.

In [4]:
import spacy

en_nlp = spacy.load('en')
de_nlp = spacy.load('de')

### Import the NLTK library

We will use the [NLTK library](NLTK) for other features we want to extract from our data. As well as use the BLUE scoring algorithm in the library

In [5]:
import nltk

In [6]:
en_s = u'Hello, world. Here are two sentences.'
de_s = u'Ich bin ein Berliner.'

### Evaluation metrics

In [7]:
def BLEU(reference,candidate,n):
    """
    Compute the BLEU score for a given candidate sentence, with respect to a
    given reference sentence.

    reference: the reference translation
    candidate: the candidate translation
    n        : the size of the ngrams
    """
    return float(
        nltk.translate.bleu_score.modified_precision([reference],candidate,n=n))

### Features

In [8]:
def pos_feature(s,nlp):
    """
    Compute the POS feature vector given a sentence and an instance of spaCy.
    The POS feature vector is a vector which indicates, per POS-tag of the
    language, what ratio of the words in the sentence have this POS-tag.

    s  : input sentence
    nlp: instance of spaCy nlp
    """
    doc       = nlp(s,tag=True,parse=False,entity=False)
    pos_count = collections.Counter([tok.tag_ for tok in doc])
    return map(lambda tag: pos_count[tag] / len(doc), nlp.tagger.tag_names)

In [9]:
# Example: print the POS-feature for an English sentence.
print(len(en_nlp.tagger.tag_names))
print(map(lambda x: round(x,2),pos_feature(en_s,en_nlp)))

56
[0.0, 0.0, 0.0, 0.0, 0.11, 0.0, 0.0, 0.22, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11, 0.0, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
# Example: print the POS-feature for a German sentence.
print(len(de_nlp.tagger.tag_names))
print(map(lambda x: round(x,2),pos_feature(de_s,de_nlp)))

57
[0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
# Example: print the first 24 values in the sentence vector for an English sentence.
en_doc = en_nlp(en_s)
print(len(en_doc.vector))
print(en_doc.vector[:24])

300
[ 0.07028833 -0.00575205  0.02307617  0.0293571   0.0150436   0.03114259
  0.00853428  0.00191485  0.00394045 -0.02803375  0.04562261 -0.00354294
 -0.04505194  0.0158382  -0.01503225 -0.06313571 -0.06844621 -0.06096174
 -0.0268664  -0.0046172  -0.031034   -0.00546252  0.0032963  -0.04281867]


In [12]:
# Example: print the first 24 values in the sentence vector for a German sentence.
de_doc = de_nlp(de_s)
print(len(de_doc.vector))
print(de_doc.vector[:24])

300
[ 0.04531119 -0.0290378   0.23427622  0.17914079 -0.19891641  0.1339044
 -0.16393779 -0.40263137 -0.04548381 -0.20328541  0.22776499 -0.1040916
 -0.1286144   0.078522    0.16846181 -0.33073679  0.34232721 -0.29092079
 -0.17509019  0.0541686   0.0833232  -0.27929959  0.0028818   0.15957041]


### Loading the data

In [13]:
# Create constants for the paths to all data files.
DATA_DIR         = os.path.abspath(os.path.join('..','data'))
BASELINE_WEIGHTS = os.path.join(DATA_DIR,'baseline.weights')
DEV_BEST         = os.path.join(DATA_DIR,'nlp2-dev.1000best.gz')
DEV_DE           = os.path.join(DATA_DIR,'nlp2-dev.de.gz')
DEV_EN_PLF       = os.path.join(DATA_DIR,'nlp2-dev.en.pw.plf-100.gz')
DEV_EN           = os.path.join(DATA_DIR,'nlp2-dev.en.s.gz')
TEST_BEST        = os.path.join(DATA_DIR,'nlp2-test.1000best.gz')
TEST_DE          = os.path.join(DATA_DIR,'nlp2-test.de.gz')
TEST_EN_PLF      = os.path.join(DATA_DIR,'nlp2-test.en.pw.plf-100.gz')
TEST_EN          = os.path.join(DATA_DIR,'nlp2-test.en.s.gz')

In [14]:
# Load the development data.
with gzip.open(DEV_EN,  'r') as f: inputs     = f.readlines()
with gzip.open(DEV_DE,  'r') as f: references = f.readlines()
with gzip.open(DEV_BEST,'r') as f: raw_candidates = f.readlines()

In [15]:
def parse_candidate(s):
    """
    Parse a candidate translation (a line from the 1000-best files) into
    a tuple containing (in order):
    
        k:              the 0-based sentence id           (int)
        source:         the source sentence               (str)
        target:         the translated sentence           (str)
        segments:       the segments and their alignments (list[(str,(int,int))])
        feature_vector: the feature vector                ({str: list[float]})
        score:          the score assigned by MOSES       (float)
        alignments:     the alignments                    ([(int,int)])

    Note: alignments in the "segments" field are pairs of states in the
    input lattice, whereas the alignments in the "alignments" field are
    pairs of a state in the input lattice together with the position of
    the output word.
    """
    k, segments_and_alignments, feature_vector, score, alignments, source = s.split(' ||| ')
    
    # Parse an id as an integer
    k = int(k)
    
    # Parse a candidate translation (with alignments) into a sentence.
    segments_and_alignments = map(lambda s: s.strip(),
                                  re.split(r'\|(\d+\-\d+)\|', segments_and_alignments))
    segments = segments_and_alignments[0::2]
    target = ' '.join(segments)
    
    # Parse a candidate translation (with alignments) into a list of segments.
    segment_alignments = map(lambda s: tuple(map(int,s.strip().split('-'))), 
                             segments_and_alignments[1::2])
    segments = zip(segments,segment_alignments)
    
    # Parse a feature vector string into a dictionary.
    feature_vector = re.split(r'([A-Za-z]+0?)=', feature_vector)
    feature_names  = feature_vector[1::2]
    feature_values = map(lambda s: map(float,s.strip().split()), feature_vector[2::2])
    feature_map = dict(zip(feature_names,feature_values))
    
    # Parse a score as a float.
    score = float(score)
    
    # Parse an alignment string into a list of tuples.
    alignments = map(lambda s: tuple(map(int,s.split('-'))), alignments.strip().split(' '))
    
    return (k, source, target, segments, feature_map, score, alignments)

In [16]:
# Example: print all relevant information for sentence with id #2.
print(inputs[2])
print(references[2])
(k, source, target, segments, feature_map, score, alignments) = parse_candidate(raw_candidates[1999])
# print(segments)
print(feature_map.values())
# print(score)

two sets of lights so close to one another : intentional or just a silly error ?

Zwei Anlagen so nah beieinander : Absicht oder Schildbürgerstreich ?

[[-200.0], [-197.749], [24.0], [-46.6874, -49.1628, -45.9683, -57.3], [0.906734, 0.017208, 0.017208], [0.0, 0.0, 0.0, 0.0], [-32.0], [-247.261], [0.0]]


### Constructing feature vector for two sentences

In [17]:
def feature_vector(e, c1, c2):
    (_, _, t1, _, f1, _, _) = c1
    (_, _, t2, _, f2, _, _) = c2
    
    return sum(f1.values(), []) + sum(f2.values(), [])
    

In [18]:
def training_label(ref, c1, c2):
    
    (_, _, t1, _, _, _, _) = c1
    (_, _, t2, _, _, _, _) = c2
    
    if (BLEU(ref, t1, 4) > BLEU(ref, t2, 4)):
        return 1
    
    return 0

### The PRO algorithm

In [19]:
def pro_corpus(inputs, references, candidates, n=0, sample_size=10):
    
    training_data = []
    
    for i, e in enumerate(inputs):
        if n == 0 or i < n: 
            g = references[i]
            c = [parse_candidate(x) for x in candidates[i:(i + 1000)]]
            training_data = training_data + pro(e, g, c, sample_size)
        
    return training_data
        
        

In [20]:
def pro(e, g, c, sample_size=10):
    
    training_data = []
    
    for i in range(0,sample_size):
        
        # Randomly pick two candidates that are not the same
        j1 = j2 = random.randint(0,999)
        while j1 == j2:
            j2 = random.randint(0,999)
            
        training_example = ( feature_vector(e,c[j1],c[j2]), training_label(g,c[j1],c[j2]) )
        
        training_data.append(training_example)
    
    return training_data
        

### Train SVM 

In [21]:
training_data = pro_corpus(inputs, references, raw_candidates, 50, 10)
(x,y) = zip(*training_data)
x = list(x)
y = list(y)

NameError: name 'candidates' is not defined

In [ ]:
from sklearn import svm

clf = svm.SVR()
clf.fit(x, y) 